In [ ]:
%pip install ipywidgets
%pip install tensorflow_datasets

In [24]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

def preprocess(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    return x, x

(ds_train, ds_test), ds_info = tfds.load(
    'smallnorb',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

ds_train = ds_train.map(preprocess)
ds_test = ds_test.map(preprocess)

input_shape = (96, 96, 1)
encoding_dimensions = 2

encoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=input_shape),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=encoding_dimensions, activation=tf.keras.activations.relu)
])

decoder = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(encoding_dimensions,)),
    tf.keras.layers.Dense(units=np.prod(input_shape), activation=tf.keras.activations.relu),
    tf.keras.layers.Reshape(target_shape=input_shape)
])

input = tf.keras.layers.Input(shape=input_shape)

output = decoder(encoder(input))

model = tf.keras.Model(inputs=input, outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(), loss=tf.keras.losses.Huber())

model.summary()

model.fit(
        ds_train.batch(32),
        epochs=1,
        batch_size=32,
        shuffle=True,
        validation_data=ds_test,
    )

# last_loss = float('inf')
# last_val_loss = float('inf')
# threshold = 0.8  # 20% improvement threshold
# patience = 10

# while (patience > 0):
#     patience -= 1
#     history = model.fit(
#         ds_train.batch(32),
#         epochs=1,
#         batch_size=32,
#         shuffle=True,
#         validation_data=ds_test,
#     )
#     loss = history.history['loss'][-1]
#     val_loss = history.history['val_loss'][-1]
#     if loss < last_loss * threshold or val_loss < last_val_loss * threshold:
#         last_loss = loss
#         last_val_loss = val_loss
#     else:
#         break


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_45 (InputLayer)       [(None, 96, 96, 1)]       0         
                                                                 
 sequential_28 (Sequential)  (None, 2)                 18434     
                                                                 
 sequential_29 (Sequential)  (None, 96, 96, 1)         27648     
                                                                 
Total params: 46,082
Trainable params: 46,082
Non-trainable params: 0
_________________________________________________________________
758/760 [============================>.] - ETA: 0s - loss: 0.1520

ValueError: in user code:

    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\engine\training.py", line 1788, in test_step
        y_pred = self(x, training=False)
    File "c:\Users\Frederik\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

test_samples = np.array([
    image
    for image, _ in tfds.as_numpy(ds_test.take(5))
])

encoded_samples = encoder.predict(test_samples)
decoded_samples = decoder.predict(encoded_samples)

n = len(test_samples)
plt.figure(figsize=(n * 2, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(test_samples[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_samples[i])
    plt.title("decoded")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()